Large language models are `stateless`. Each call is independent.
They only appear to have memory by providing each call with a **context**. This is what gives them memory.

In [48]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationTokenBufferMemory
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models.google_palm import ChatGooglePalm

In [2]:
import os
api_key = os.getenv("PALM_API_KEY")

### Candidate Count Errors
There is a potential error that langchain could raise when you ask for a prediction. This is `ChatGooglePalmError`. The reason this is happening is that when temperature is set to low values, the llm returns `None`. Hence why it says: **ChatResponse must have at least one candidate**. 

> candidate_count: The **maximum** number of generated response messages to return.
    
            This value must be between `[1, 8]`, inclusive. If unset, this
            will default to `1`.
    
            Note: Only unique candidates are returned. Higher temperatures are more
            likely to produce unique candidates. Setting `temperature=0.0` will always
            return 1 candidate regardless of the `candidate_count`.

The patch for now is to set temperature high.

Also setting k to high seems to also mitigate this error.

### k (parameter)
Determines how much is remembered. It takes an integer specifying the number of **conversational exchanges** to be remembered.
By default `k=5`

### ConversationBufferWindow
Provides a window of context.

In [71]:
llm = ChatGooglePalm(google_api_key=api_key, temperature=0.7)

In [75]:

# Initialize ConversationBufferWindowMemory
memory = ConversationBufferMemory()
memory.save_context({"input": "Hi"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})

# Load memory variables
memory.load_memory_variables({})


{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

In [76]:
# Create ConversationChain
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

In [78]:
conversation.predict(input="My name is Andrew?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi
AI: What's up
Human: Not much, just hanging
AI: Cool
Human: What is my name?
AI: I do not know your name. I am a large language model, also known as a conversational AI or chatbot trained to be informative and comprehensive. I am trained on a massive amount of text data, and I am able to communicate and generate human-like text in response to a wide range of prompts and questions. For example, I can provide summaries of factual topics or create stories.

However, I am not able to access your personal information, such as your name. This is because I am not trained on personal data, and I am not able to track or store personal information. I am only able to ac

"Nice to meet you Andrew! It's a pleasure to chat with you."

### ConversationTokenBufferMemory
Allows us to specify a max token value.

Requires the **transformers** library to work. Install it, if already not installed using, `pip install transformers`

In [54]:
llm = ChatGooglePalm(google_api_key=api_key, temperature=0.7)

In [55]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=100)

In [57]:
memory.save_context({"input": "Hi"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
memory.load_memory_variables({})

{'history': "AI: Hello, Andrew! It's nice to meet you. How can I help you today?\nHuman: What is 1+1?\nAI: Hi Andrew, it's nice to meet you too!\r\n\r\n1+1 is 2.\nHuman: What is my name?\nAI: Hi Andrew! It's nice to meet you again.\r\n\r\nYour name is Andrew.\nHuman: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

In [ ]:

conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

In [58]:
conversation.predict(input="Hi, my name is Andrew")
conversation.predict(input="What is 1+1?")
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
AI: Hello, Andrew! It's nice to meet you. How can I help you today?
Human: What is 1+1?
AI: Hi Andrew, it's nice to meet you too!

1+1 is 2.
Human: What is my name?
AI: Hi Andrew! It's nice to meet you again.

Your name is Andrew.
Human: Hi
AI: What's up
Human: Not much, just hanging
AI: Cool
Human: Hi, my name is Andrew
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
AI: Hi A

"Hello Andrew, it's nice to meet you again. What can I help you with today?"

### ConversationSummaryBufferMemory

In [65]:
llm = ChatGooglePalm(google_api_key=api_key, temperature=0.7)

In [66]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=50)

In [67]:
# Create a long string
schedule = "There is a meeting at 8am with your product team. " \
           "You will need your PowerPoint presentation prepared. " \
           "9am-12pm have time to work on your LangChain " \
           "project which will go quickly because Langchain is such a powerful tool. " \
           "At Noon, lunch at the Italian restaurant with a customer who is driving " \
           "from over an hour away to meet you to understand the latest in AI. " \
           "Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, {"output": f"{schedule}"})
memory.load_memory_variables({})

{'history': "System: Human: Hello\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool\nHuman: What is on the schedule today?\nAI: Today, we will be discussing the future of artificial intelligence. We will explore the potential benefits and risks of AI, and discuss how we can ensure that AI is used for good.\nHuman: That sounds interesting. I'm looking forward to it.\nAI: Me too.\n\nThe human and AI discuss the future of artificial intelligence. They explore the potential benefits and risks of AI, and discuss how we can ensure that AI is used for good. The human is interested in the topic, and the AI is knowledgeable and engaging. The conversation is productive and informative.\nAI: There is a meeting at 8am with your product team. You will need your PowerPoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the Italian restaurant with a customer who is driving from over a

In [69]:
conversation = ConversationChain(llm=llm, memory=memory,verbose=True)

In [70]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: Human: Hello
AI: What's up
Human: Not much, just hanging
AI: Cool
Human: What is on the schedule today?
AI: Today, we will be discussing the future of artificial intelligence. We will explore the potential benefits and risks of AI, and discuss how we can ensure that AI is used for good.
Human: That sounds interesting. I'm looking forward to it.
AI: Me too.

The human and AI discuss the future of artificial intelligence. They explore the potential benefits and risks of AI, and discuss how we can ensure that AI is used for good. The human is interested in the topic, and the AI is knowledgeable and engaging. The conversation is productive and informative.
AI: Ther

"There are many different types of LLM demos that you could show your customer. Here are a few ideas:\n\n* You could show a demo of the LLM being used to generate text. For example, you could ask the LLM to write a poem, a news article, or a blog post.\n* You could show a demo of the LLM being used to translate text from one language to another. For example, you could ask the LLM to translate a sentence from English to Spanish or vice versa.\n* You could show a demo of the LLM being used to answer questions. For example, you could ask the LLM questions about a particular topic and see how well it can answer them.\n* You could show a demo of the LLM being used to generate creative content. For example, you could ask the LLM to write a song, a story, or a piece of art.\n\nThe best demo to show will depend on your customer's interests and needs. If you are not sure what to show, you could always ask your customer what they would like to see."

### Steps
The procedure of providing memory to the llm is as follows:

1. Initialize the llm
`llm = ChatGooglePalm(google_api_key=api_key, temperature=0.7)`

2. Initialize memory buffer
`memory = ConversationBufferMemory()`

3. Add context to memory, just initlialized
```
memory.save_context({"input": "Hi"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
```

4. Initialize conversation chain
`conversation = ConversationChain(llm=llm, memory=memory,verbose=True)`

5. Ask the llm for a response.
`conversation.predict(input="What is my name?")`

**NB**: It helps greatly if what you ask is related to the context you provided.

### Conclusions

Google Palm is quite finicky as it will sometimes return `None`, rather than a text response which causes the langchain wrapper to raise a `ChatGooglePalmError`.

We can reduce the chances of getting no response by making sure the context is sufficient for the llm to make a response.
We do this by:

1. Setting temperature to high
2. Setting k to a hiigher value (That way it remembers more)